# T1. dataset 和 vocabulary 的基本使用

&emsp; 1 &ensp; dataset 的使用与结构
 
&emsp; &emsp; 1.1 &ensp; dataset 的结构与创建

&emsp; &emsp; 1.2 &ensp; dataset 的数据预处理

&emsp; &emsp; 1.3 &ensp; 延伸：instance 和 field

&emsp; 2 &ensp; vocabulary 的结构与使用

&emsp; &emsp; 2.1 &ensp; vocabulary 的创建与修改

&emsp; &emsp; 2.2 &ensp; vocabulary 与 OOV 问题

&emsp; 3 &ensp; dataset 和 vocabulary 的组合使用
 
&emsp; &emsp; 3.1 &ensp; 从 dataframe 中加载 dataset

&emsp; &emsp; 3.2 &ensp; 从 dataset 中获取 vocabulary

## 1. dataset 的基本使用

### 1.1  dataset 的结构与创建

在`fastNLP 0.8`中，使用`DataSet`模块表示数据集，**`dataset`类似于关系型数据库中的数据表**（下文统一为小写`dataset`）

&emsp; **主要包含`field`字段和`instance`实例两个元素**，对应`table`中的`field`字段和`record`记录

在`fastNLP 0.8`中，`DataSet`模块被定义在`fastNLP.core.dataset`路径下，导入该模块后，最简单的

&emsp; 初始化方法，即将字典形式的表格 **`{'field1': column1, 'field2': column2, ...}`** 传入构造函数

In [1]:
from fastNLP.core.dataset import DataSet

data = {'idx': [0, 1, 2],  
        'sentence':["This is an apple .", "I like apples .", "Apples are good for our health ."],
        'words': [['This', 'is', 'an', 'apple', '.'], 
                  ['I', 'like', 'apples', '.'], 
                  ['Apples', 'are', 'good', 'for', 'our', 'health', '.']],
        'num': [5, 4, 7]}

dataset = DataSet(data)
print(dataset)

+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 1   | I like apples .        | ['I', 'like', 'appl... | 4   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+


&emsp; 在`dataset`的实例中，字段`field`的名称和实例`instance`中的字符串也可以中文

In [2]:
temp = {'序号': [0, 1, 2],  
        '句子':["生活就像海洋，", "只有意志坚强的人，", "才能到达彼岸。"],
        '字符': [['生', '活', '就', '像', '海', '洋', '，'], 
                  ['只', '有', '意', '志', '坚', '强', '的', '人', '，'], 
                  ['才', '能', '到', '达', '彼', '岸', '。']],
        '长度': [7, 9, 7]}

chinese = DataSet(temp)
print(chinese)

+------+--------------------+------------------------+------+
| 序号 | 句子               | 字符                   | 长度 |
+------+--------------------+------------------------+------+
| 0    | 生活就像海洋，     | ['生', '活', '就', ... | 7    |
| 1    | 只有意志坚强的人， | ['只', '有', '意', ... | 9    |
| 2    | 才能到达彼岸。     | ['才', '能', '到', ... | 7    |
+------+--------------------+------------------------+------+


在`dataset`中，使用`drop`方法可以删除满足条件的实例，这里使用了python中的`lambda`表达式

&emsp; 注一：在`drop`方法中，通过设置`inplace`参数将删除对应实例后的`dataset`作为一个新的实例生成

In [3]:
dropped = dataset
dropped = dropped.drop(lambda ins:ins['num'] < 5, inplace=False)
print(id(dropped), id(dataset))
print(dropped)
print(dataset)

1608199516936 1607874531400
+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+
+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 1   | I like apples .        | ['I', 'like', 'appl... | 4   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+


&emsp; 注二：**对对象使用等号一般表示传引用**，所以对`dataset`使用等号，是传引用而不是赋值

&emsp; &emsp; 如下所示，**`dropped`和`dataset`具有相同`id`**，**对`dropped`执行删除操作`dataset`同时会被修改**

In [4]:
dropped = dataset
dropped.drop(lambda ins:ins['num'] < 5)
print(id(dropped), id(dataset))
print(dropped)
print(dataset)

1607874531400 1607874531400
+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+
+-----+------------------------+------------------------+-----+
| idx | sentence               | words                  | num |
+-----+------------------------+------------------------+-----+
| 0   | This is an apple .     | ['This', 'is', 'an'... | 5   |
| 2   | Apples are good for... | ['Apples', 'are', '... | 7   |
+-----+------------------------+------------------------+-----+


在`dataset`中，使用`delet_instance`方法可以删除对应序号的`instance`实例，序号从0开始

In [5]:
dataset = DataSet(data)
dataset.delete_instance(2)
print(dataset)

+-----+--------------------+------------------------+-----+
| idx | sentence           | words                  | num |
+-----+--------------------+------------------------+-----+
| 0   | This is an apple . | ['This', 'is', 'an'... | 5   |
| 1   | I like apples .    | ['I', 'like', 'appl... | 4   |
+-----+--------------------+------------------------+-----+


在`dataset`中，使用`delet_field`方法可以删除对应名称的`field`字段

In [6]:
dataset.delete_field('num')
print(dataset)

+-----+--------------------+------------------------------+
| idx | sentence           | words                        |
+-----+--------------------+------------------------------+
| 0   | This is an apple . | ['This', 'is', 'an', 'app... |
| 1   | I like apples .    | ['I', 'like', 'apples', '... |
+-----+--------------------+------------------------------+


### 1.2  dataset 的数据预处理

在`dataset`模块中，`apply`、`apply_field`、`apply_more`和`apply_field_more`函数可以进行简单的数据预处理

&emsp; **`apply`和`apply_more`输入整条实例**，**`apply_field`和`apply_field_more`仅输入实例的部分字段**

&emsp; **`apply`和`apply_field`仅输出单个字段**，**`apply_more`和`apply_field_more`则是输出多个字段**

&emsp; **`apply`和`apply_field`返回的是个列表**，**`apply_more`和`apply_field_more`返回的是个字典**

***

`apply`的参数包括一个函数`func`和一个新字段名`new_field_name`，函数`func`的处理对象是`dataset`模块中

&emsp; 的每个`instance`实例，函数`func`的处理结果存放在`new_field_name`对应的新建字段内

In [ ]:
data = {'idx': [0, 1, 2], 
        'sentence':["This is an apple .", "I like apples .", "Apples are good for our health ."], }
dataset = DataSet(data)
dataset.apply(lambda ins: ins['sentence'].split(), new_field_name='words')
print(dataset)

Output()

&emsp; **`apply`使用的函数可以是一个基于`lambda`表达式的匿名函数**，**也可以是一个自定义的函数**

In [ ]:
dataset = DataSet(data)

def get_words(instance):
    sentence = instance['sentence']
    words = sentence.split()
    return words

dataset.apply(get_words, new_field_name='words')
print(dataset)

`apply_field`的参数，除了函数`func`外还有`field_name`和`new_field_name`，该函数`func`的处理对象仅

&emsp; 是`dataset`模块中的每个`field_name`对应的字段内容，处理结果存放在`new_field_name`对应的新建字段内

In [ ]:
dataset = DataSet(data)
dataset.apply_field(lambda sent:sent.split(), field_name='sentence', new_field_name='words')
print(dataset)

`apply_more`的参数只有函数`func`，函数`func`的处理对象是`dataset`模块中的每个`instance`实例

&emsp; 要求函数`func`返回一个字典，根据字典的`key-value`确定存储在`dataset`中的字段名称与内容

In [ ]:
dataset = DataSet(data)
dataset.apply_more(lambda ins:{'words': ins['sentence'].split(), 'num': len(ins['sentence'].split())})
print(dataset)

`apply_more`的参数只有函数`func`，函数`func`的处理对象是`dataset`模块中的每个`instance`实例

&emsp; 要求函数`func`返回一个字典，根据字典的`key-value`确定存储在`dataset`中的字段名称与内容

In [ ]:
dataset = DataSet(data)
dataset.apply_field_more(lambda sent:{'words': sent.split(), 'num': len(sent.split())}, 
                         field_name='sentence')
print(dataset)

### 1.3 延伸：instance 和 field

在`fastNLP 0.8`中，使用`Instance`模块表示数据集`dataset`中的每条数据，被称为实例

&emsp; 构造方式类似于构造一个字典，通过键值相同的`Instance`列表，也可以初始化一个`dataset`，代码如下

In [ ]:
from fastNLP.core.dataset import DataSet
from fastNLP.core.dataset import Instance

dataset = DataSet([
    Instance(sentence="This is an apple .",
        words=['This', 'is', 'an', 'apple', '.'],
        num=5),
    Instance(sentence="I like apples .",
        words=['I', 'like', 'apples', '.'],
        num=4),
    Instance(sentence="Apples are good for our health .",
        words=['Apples', 'are', 'good', 'for', 'our', 'health', '.'],
        num=7),
    ])

&emsp; 通过`items`、`keys`和`values`方法，可以分别获得`dataset`的`item`列表、`key`列表、`value`列表

In [ ]:
ins = Instance(sentence="This is an apple .", words=['This', 'is', 'an', 'apple', '.'], num=5)

print(ins.items())
print(ins.keys())
print(ins.values())

&emsp; 通过`add_field`方法，可以在`Instance`实例中，通过参数`field_name`添加字段，通过参数`field`赋值

In [ ]:
ins.add_field(field_name='idx', field=0)
print(ins)

在`fastNLP 0.8`中，使用`FieldArray`模块表示数据集`dataset`中的每条字段名（注：没有`field`类）

&emsp; 通过`get_all_fields`方法可以获取`dataset`的字段列表

&emsp; 通过`get_field_names`方法可以获取`dataset`的字段名称列表，代码如下

In [ ]:
dataset.get_all_fields()

In [ ]:
dataset.get_field_names()

其他`dataset`的基本使用：通过`in`或者`has_field`方法可以判断`dataset`的是否包含某种字段

&emsp; 通过`rename_field`方法可以更改`dataset`中的字段名称；通过`concat`方法可以实现两个`dataset`中的拼接

&emsp; 通过`len`可以统计`dataset`中的实例数目；`dataset`的全部变量与函数可以通过`dir(dataset)`查询

In [ ]:
print(len(dataset), dataset.has_field('length')) 
if 'num' in dataset:
    dataset.rename_field('num', 'length')
elif 'length' in dataset:
    dataset.rename_field('length', 'num')
dataset.concat(dataset)
print(len(dataset), dataset.has_field('length'))  
print(dataset)  

## 2. vocabulary 的结构与使用

### 2.1 vocabulary 的创建与修改

在`fastNLP 0.8`中，使用`Vocabulary`模块表示词汇表，**`vocabulary`的核心是从单词到序号的映射**

&emsp; 可以直接通过构造函数实例化，通过查找`word2idx`属性，可以找到`vocabulary`映射对应的字典实现

&emsp; **默认补零`padding`用`<pad>`表示**，**对应序号为0**；**未知单词`unknown`用`<unk>`表示**，**对应序号1**

&emsp; 通过打印`vocabulary`可以看到词汇表中的单词列表，其中，`padding`和`unknown`不会显示

In [ ]:
from fastNLP.core.vocabulary import Vocabulary

vocab = Vocabulary()
print(vocab)
print(vocab.word2idx)
print(vocab.padding, vocab.padding_idx)
print(vocab.unknown, vocab.unknown_idx)

在`vocabulary`中，通过`add_word`方法或`add_word_lst`方法，可以单独或批量添加单词

&emsp; 通过`len`或`word_count`属性，可以显示`vocabulary`的单词量和每个单词添加的次数

In [ ]:
vocab.add_word_lst(['生活', '就像', '海洋'])
print(len(vocab), vocab.word_count)
vocab.add_word('只有')
print(len(vocab), vocab.word_count)
print(len(vocab), vocab.word2idx)

&emsp; **通过`to_word`方法可以找到单词对应的序号**，**通过`to_index`方法可以找到序号对应的单词**

&emsp; &emsp; 由于序号0和序号1已经被占用，所以**新加入的词的序号从2开始计数**，如`'生活'`对应2

&emsp; &emsp; 通过`has_word`方法可以判断单词是否在词汇表中，没有的单词被判做`<unk>`

In [ ]:
print(vocab.to_word(0), vocab.to_index('<pad>'))
print(vocab.to_word(1), vocab.to_index('<unk>'))
print(vocab.to_word(2), vocab.to_index('生活'))
print('彼岸', vocab.to_index('彼岸'), vocab.has_word('彼岸'))

**`vocabulary`允许反复添加相同单词**，**可以通过`word_count`方法看到相应单词被添加的次数**

&emsp; 但其中没有`<unk>`和`<pad>`，`vocabulary`的全部变量与函数可以通过`dir(vocabulary)`查询

&emsp; 注：**使用`add_word_lst`添加单词**，**单词对应序号不会动态调整**，**使用`dataset`添加单词的情况不同**

In [ ]:
vocab.add_word_lst(['生活', '就像', '海洋', '只有', '意志', '坚强的', '人', '人', '人', '人', '才', '能', '到达', '彼岸'])
print(vocab.to_word(2), vocab.to_index('生活'))
print('彼岸', vocab.to_index('彼岸'), vocab.has_word('彼岸'))
print(len(vocab), vocab.word_count)
print(len(vocab), vocab.word2idx)

### 2.2 vocabulary 与 OOV 问题

在`vocabulary`模块初始化的时候，可以通过指定`unknown`和`padding`为`None`，限制其存在

&emsp; 此时添加单词直接从0开始标号，如果遇到未知单词会直接报错，即 out of vocabulary

In [ ]:
vocab = Vocabulary(unknown=None, padding=None)

vocab.add_word_lst(['positive', 'negative'])
print(vocab.word2idx)

try:
    print(vocab.to_index('neutral'))
except ValueError:
    print("ValueError: word `neutral` not in vocabulary")

&emsp; 相应的，如果只指定其中的`unknown`，则编号会后移一个，同时遇到未知单词全部当做`<unk>`

In [ ]:
vocab = Vocabulary(unknown='<unk>', padding=None)

vocab.add_word_lst(['positive', 'negative'])
print(vocab.word2idx)

print(vocab.to_index('neutral'), vocab.to_word(vocab.to_index('neutral')))

## 3 dataset 和 vocabulary 的组合使用
 
### 3.1 从 dataframe 中加载 dataset

以下通过 [NLP-beginner](https://github.com/FudanNLP/nlp-beginner) 实践一中 [Rotten Tomatoes 影评数据集](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews) 的部分训练数据组成`test4dataset.tsv`文件

&emsp; 介绍如何使用`dataset`、`vocabulary`简单加载并处理数据集，首先使用`pandas`模块，读取原始数据的`dataframe`

In [ ]:
import pandas as pd

df = pd.read_csv('./data/test4dataset.tsv', sep='\t')
df

接着，通过`dataset`中的`from_pandas`方法填充数据集，并使用`apply_more`方法对文本进行分词操作

In [ ]:
from fastNLP.core.dataset import DataSet

dataset = DataSet()
dataset = dataset.from_pandas(df)
dataset.apply_more(lambda ins:{'SentenceId': ins['SentenceId'], 
                               'Sentence': ins['Sentence'].lower().split(), 'Sentiment': ins['Sentiment']})
print(dataset)

&emsp; 如果需要保存中间结果，也可以使用`dataset`的`to_csv`方法，生成`.csv`或`.tsv`文件

In [ ]:
dataset.to_csv('./data/test4dataset.csv')

### 3.2 从 dataset 中获取 vocabulary

然后，初始化`vocabulary`，使用`vocabulary`中的`from_dataset`方法，从`dataset`的指定字段中

&emsp; 获取字段中的所有元素，然后编号；如果指定字段是个列表，则针对字段中所有列表包含的元素编号

&emsp; 注：**使用`dataset`添加单词**，**不同于`add_word_list`**，**单词被添加次数越多**，**序号越靠前**，例如案例中的`a`

In [ ]:
from fastNLP.core.vocabulary import Vocabulary

vocab = Vocabulary()
vocab = vocab.from_dataset(dataset, field_name='Sentence')
print(vocab.word_count, '\n')
print(vocab.word2idx, '\n')
print(vocab)

之后，**通过`vocabulary`的`index_dataset`方法**，**调整`dataset`中指定字段的元素**，**使用编号将之代替**

&emsp; 使用上述方法，可以将影评数据集中的单词序列转化为词编号序列，为接下来转化为词嵌入序列做准备

In [ ]:
vocab.index_dataset(dataset, field_name='Sentence')
print(dataset)

最后，使用相同方法，再将`dataset`中`Sentiment`字段中的`negative`、`neutral`、`positive`转化为数字编号

In [ ]:
target_vocab = Vocabulary(padding=None, unknown=None)

target_vocab.from_dataset(dataset, field_name='Sentiment')
print(target_vocab.word2idx)
target_vocab.index_dataset(dataset, field_name='Sentiment')
print(dataset)

在最后的最后，通过以下的一张图，来总结本章关于`dataset`和`vocabulary`主要知识点的讲解，以及两者的联系

<img src="./figures/T1-fig-dataset-and-vocabulary.png" width="80%" height="80%" align="center"></img>